In [1]:
from msedge.selenium_tools import Edge, EdgeOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.common.exceptions import NoSuchElementException
import os
import time

In [2]:
def wait_until(browser, xpath):
    WebDriverWait(browser, 100).until(
        expected_conditions.presence_of_element_located((By.XPATH, xpath)))

In [3]:
def scrap_memes(number_of_memes, group):
    options = EdgeOptions()
    options.use_chromium = True
    # options.add_argument('--headless')
    options.add_argument('--disable-notifications')

    options.add_experimental_option("prefs", {
        "download.default_directory":  os.getcwd() + '\memes',
        "download.prompt_for_download": "false",
        "download.directory_upgrade": "true",
        "safebrowsing.enabled": "true"})

    browser = Edge('msedgedriver.exe', options=options)
    browser.get('https://web.whatsapp.com/')

    # find de group
    title = group
    text_box = '//div[@role="textbox"]'
    wait_until(browser, text_box)
    title_group = f'//span[@title="{title}"]'
    data = browser.find_elements(By.XPATH, title_group)
    if not data:
        # search the group
        browser.find_element(By.XPATH, '//div[@role="textbox"]').send_keys(title)

    # access to the group
    wait_until(browser, title_group)
    browser.find_element(By.XPATH, title_group).click()

    # access to the description of the group
    wait_until(browser, '//span[@data-testid="search-alt"]')
    browser.find_elements(By.XPATH, title_group)[1].click()

    # access to the gallery of the group
    chevron_right = '//span[@data-testid="chevron-right-alt"]'
    wait_until(browser, chevron_right)
    time.sleep(1)
    browser.find_element(By.XPATH, chevron_right).click()

    # access to the first image from the gallery
    first_image = '//*[@class="_23fpc"]'
    wait_until(browser, first_image)
    browser.find_element(By.XPATH, first_image).click()
    for _ in range(number_of_memes):
        try:
            wait_until(browser, '//span[@data-icon="download"]')
            browser.find_element(By.XPATH, '//video')
            browser.find_element(By.XPATH, '//span[@data-icon="chevron-left"]').click()
        except NoSuchElementException:
            browser.find_element(By.XPATH, '//span[@data-icon="download"]').click()
            browser.find_element(By.XPATH, '//span[@data-icon="chevron-left"]').click()

In [4]:
scrap_memes(50, 'Free memes')